In [2]:
import wandb
import cuda

ModuleNotFoundError: No module named 'cuda'

In [7]:
import torch
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline



# 1. Load your dataset
# Assuming it's in the 'conll2003' format for simplicity
def process_conll_data(example):
    # Split lines and filter out empty lines
    lines = [line.strip() for line in example['text'].split('\n') if line.strip()]

    # Split each line into its components
    tokens, ner_tags = [], []
    for line in lines:
        parts = line.split()
        if len(parts) == 2:  # Ensure that each line has exactly 2 parts
            tokens.append(parts[0])
            ner_tags.append(parts[1])
        else:
            print(f"Skipping line with unexpected format: {line}")

    return {
        'tokens': tokens,
        'ner_tags': ner_tags
    }
    
data_files = {
    'train': 'train.txt',
    'validation': 'val.txt',
    'test': 'test.txt'
}

dataset = load_dataset('text', data_files=data_files)
processed_datasets = dataset.map(process_conll_data)

# 2. Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

# 3. Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["tokens"], truncation=True, padding="max_length", is_split_into_words=True)

tokenized_datasets = processed_datasets.map(tokenize_function, batched=True)

# 4. Define a data collator to handle token-level tasks (like NER)
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

# 5. Define the model
#model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(dataset["train"].features["ner_tags"].feature))
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER", num_labels=len(dataset["train"].features["ner_tags"].feature))


import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 6. Prepare for training
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    do_train=True,
    do_eval=True,
    output_dir="./results",
)

# 7. Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# 8. Train the model
trainer.train()

# 9. Save the model
model.save_pretrained("./ner_model")
tokenizer.save_pretrained("./ner_model")


Map:   0%|          | 0/6821406 [00:00<?, ? examples/s]

KeyboardInterrupt: 